In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

import os
import sys

import tensorflow as tf 

import yfinance as yf 

#    Fetching the stock data

Price and volume

In [2]:
data = yf.Ticker("AAPL")

ohlc = data.history(period="max", interval="1d")[['Open', 'High', 'Low', 'Close', 'Volume']]
ohlc.index = ohlc.index.tz_localize(None)  # Make index timezone-naive
volume = ohlc['Volume']

Latest quaterly financials 

In [3]:
dividends = data.dividends
splits = data.splits
actions = data.actions
capital_gains = data.capital_gains

income_stmt = data.income_stmt
balance_sheet = data.balance_sheet
cashflow = data.cash_flow
tmm_cashflow = data.ttm_cash_flow
earnings = data.earnings

c:\Users\dawid\Desktop\BSEPrediction\.venv\Lib\site-packages\yfinance\scrapers\fundamentals.py:36: DeprecationWarning: 'Ticker.earnings' is deprecated as not available via API. Look for "Net Income" in Ticker.income_stmt.
  warnings.warn("'Ticker.earnings' is deprecated as not available via API. Look for \"Net Income\" in Ticker.income_stmt.", DeprecationWarning)


Additional data

In [4]:
info = data.info
fast_info = data.fast_info
calendar = data.calendar
earnings_dates = data.earnings_dates
# sec_fillings = data.sec_fillings

HTTPError: HTTP Error 502: 

Analysis 

In [5]:
recommendations = data.recommendations
recommendations_summary = data.recommendations_summary

upgrades_downgrades = data.upgrades_downgrades
sustainability = data.sustainability

analyst_price_targets = data.analyst_price_targets
earnings_estimate = data.earnings_estimate
revenue_estimate = data.revenue_estimate

earnings_history = data.earnings_history
eps_trend = data.eps_trend
# eps_revision = data.eps_revision

Holdings

In [6]:
funds_data = data.funds_data
insider_purchases = data.insider_purchases
insider_transactions = data.insider_transactions
insider_roster_holders = data.insider_roster_holders
major_holders = data.major_holders
# institutional_holder = data.institutional_holder
mutualfund_holders = data.mutualfund_holders

# Fetching the market data

In [7]:
us_market = yf.Market('US')
gb_market = yf.Market('GB')

asian_market = yf.Market('ASIA')
european_market = yf.Market('EUROPE')

rates = yf.Market('RATES')
commodities = yf.Market('COMMODITIES')
currencies = yf.Market('CURRENCIES')
crypto = yf.Market('CRYPTOCURRENCIES')

# Sector and industry

In [8]:
print(yf.Sector.industries) # to be continued

# Data cleaning and preprocessing

In [9]:
total_dividend_through_years = dividends.sum()
total_number_of_stock_splits = len(splits)
# Process capital gains 

Financial statement transformation

In [20]:
final_df = pd.DataFrame(index=ohlc.index)

# Income Statement
income_stmt = data.financials.T  # Transpose so each row is a report
income_stmt.index = pd.to_datetime(income_stmt.index)  # Ensure datetime index

income_stmt_sorted = income_stmt.sort_index()
aligned_income = pd.DataFrame(index=ohlc.index, columns=income_stmt.columns)

for report_date in income_stmt_sorted.index:
    mask = (aligned_income.index >= report_date)
    aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
    aligned_income.loc[aligned_income.index >= report_date] = \
    pd.DataFrame([income_stmt.loc[report_date].values] * len(aligned_income.loc[aligned_income.index >= report_date]),
                 index=aligned_income.loc[aligned_income.index >= report_date].index,
                 columns=aligned_income.columns)
    
aligned_income.fillna(method='ffill', inplace=True)

# Balance Sheet
balance_sheet = data.balance_sheet.T
balance_sheet.index = pd.to_datetime(balance_sheet.index)
balance_sheet_sorted = balance_sheet.sort_index()
aligned_balance = pd.DataFrame(index=ohlc.index, columns=balance_sheet.columns)
for report_date in balance_sheet_sorted.index:
    mask = (aligned_balance.index >= report_date)
    aligned_balance.loc[mask] = aligned_balance.loc[mask].fillna(method='ffill')
    aligned_balance.loc[aligned_balance.index >= report_date] = \
        pd.DataFrame([balance_sheet.loc[report_date].values] * len(aligned_balance.loc[aligned_balance.index >= report_date]),
                     index=aligned_balance.loc[aligned_balance.index >= report_date].index,
                     columns=aligned_balance.columns)
aligned_balance.fillna(method='ffill', inplace=True)

# Cash Flow
cashflow = data.cashflow.T
cashflow.index = pd.to_datetime(cashflow.index)
cashflow_sorted = cashflow.sort_index()
aligned_cashflow = pd.DataFrame(index=ohlc.index, columns=cashflow.columns)
for report_date in cashflow_sorted.index:
    mask = (aligned_cashflow.index >= report_date)
    aligned_cashflow.loc[mask] = aligned_cashflow.loc[mask].fillna(method='ffill')
    aligned_cashflow.loc[aligned_cashflow.index >= report_date] = \
        pd.DataFrame([cashflow.loc[report_date].values] * len(aligned_cashflow.loc[aligned_cashflow.index >= report_date]),
                     index=aligned_cashflow.loc[aligned_cashflow.index >= report_date].index,
                     columns=aligned_cashflow.columns)
aligned_cashflow.fillna(method='ffill', inplace=True)

C:\Users\dawid\AppData\Local\Temp\ipykernel_14948\2446286178.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
C:\Users\dawid\AppData\Local\Temp\ipykernel_14948\2446286178.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
C:\Users\dawid\AppData\Local\Temp\ipykernel_14948\2446286178.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
C:\Users\dawid\AppData\Local\

In [25]:
# Dividends
# Align dividends to ohlc index (ensure both indexes are timezone-naive)
dividends = data.dividends.copy()
dividends.index = dividends.index.tz_localize(None)
aligned_dividends = pd.DataFrame(index=ohlc.index)
aligned_dividends['Dividends'] = dividends.reindex(ohlc.index, method='ffill').fillna(0)

# Splits
# Align splits to ohlc index (ensure both indexes are timezone-naive)
splits = data.splits.copy()
splits.index = splits.index.tz_localize(None)
aligned_splits = pd.DataFrame(index=ohlc.index)
aligned_splits['Splits'] = splits.reindex(ohlc.index, method='ffill').fillna(0)

# Combine all dataframes
combined_df = pd.concat([ohlc, aligned_income, aligned_balance, aligned_cashflow, aligned_dividends, aligned_splits], axis=1)

In [24]:
combined_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Tax Effect Of Unusual Items',
       'Tax Rate For Calcs', 'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation',
       ...
       'Change In Receivables', 'Changes In Account Receivables',
       'Other Non Cash Items', 'Stock Based Compensation', 'Deferred Tax',
       'Deferred Income Tax', 'Depreciation Amortization Depletion',
       'Depreciation And Amortization',
       'Net Income From Continuing Operations', 'Dividends'],
      dtype='object', length=166)

# Model creation

Add features like technical indicators

Random forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler